In [11]:
import re, requests, os, time, random, csv, tqdm

In [7]:
def createBookBlocks(fn= './data/GUTINDEX.ALL.iso-8859-1.txt'):
    gut_all= []
    book_block= []
    for line in open(fn):
        if not line.rstrip():  # book end
            if book_block: 
                gut_all.append(book_block)
            book_block= []
            continue
        if line.rstrip()[-1].isdigit():  # new book
            book_block.append(line.rstrip())
        elif book_block:
            book_block.append(line.rstrip())
    return gut_all

In [25]:
def getBookInd(book_block):
    return book_block[0].split()[-1]

def getBookTitle(book_block):
    return ' '.join(book_block[0].split()[:-1]).encode('ascii', 'ignore').decode()
    
def getBookText(book_ind):
    base_url= 'http://gutenberg.readingroo.ms/{path}/{ind}/'.format(path='/'.join(book_ind[:-1]), ind=book_ind)
    text_fnl= re.findall('>(\S+.txt)', requests.get(base_url).text)
    if text_fnl:
        text_url= os.path.join(base_url, text_fnl[0])
        time.sleep(random.random()/10)
        req= requests.get(text_url)
        if not req.status_code==200:
            print( req.status_code)
            print( text_url)
            print( 'book_ind:%s'%book_ind)
        return req.text.lower()
    else:
        return None



In [22]:
def getTextsMatchingKeywords(gut_all, 
                             keywords= [], 
                             outfn='./data/matchingTexts.csv', 
                             book_path_local='./data/texts'):
    books= []
    with open(outfn, 'w') as outf:
        outc= csv.DictWriter(outf, fieldnames='ind,title,mathing'.split(','))
        outc.writeheader()
        for book_block in tqdm.tqdm(gut_all):
            book_ind= getBookInd(book_block)
            book_title= getBookTitle(book_block)
            book_text= getBookText(book_ind)
            if book_text:
                matches= [s for s in keywords if s.lower() in book.text.lower()]
                if matches:
                    if book_path_local:
                        open(os.path.join(book_path_local, book_ind+'.txt'), 'w').write(book_text.encode('ascii', 'ignore').decode())
                    outc.writerow({'ind':int(book_ind), 
                                   'title':book_title, 
                                   'matching':';'.join(matches)})

In [27]:
def getTextsByAuthor(gut_all, author= 'Oscar Wilde', outfn='./data/matchingTexts.csv', book_path_local='./data/texts'):
    books= []
    with open(outfn, 'w') as outf:
        outc= csv.DictWriter(outf, fieldnames='ind,title'.split(','))
        outc.writeheader()
        for book_block in tqdm.tqdm(gut_all):
            if 'by %s'%author in book_block[0]:
                book_ind= getBookInd(book_block)
                book_title= getBookTitle(book_block)
                book_text= getBookText(book_ind)
                if book_text:
                    if book_path_local:
                        open(os.path.join(book_path_local, book_ind+'.txt'), 'w').write(book_text.encode('ascii', 'ignore').decode())
                    outc.writerow({'ind':int(book_ind), 
                                   'title':book_title})

In [14]:
gut_ind= parseIndex()

### Cheshire cat

In [5]:
keywords= set(['Cheshire %s'%i for i in 'cat,feline,tomcat,tom,kitten,mouser,puss,kitty,furball'.split(',')])

In [ ]:
getTextsMatchingKeywords(gut_ind, 
                         keywords,
                         outfn='./data/matchingTexts.csv', 
                         book_path_local='./data/texts')

### Oscar Wilde

In [28]:
getTextsByAuthor(gut_ind, 
                 author= 'Oscar Wilde', 
                 outfn='./data/matchingTexts.csv', 
                 book_path_local='./data/texts_ow')


100%|██████████| 43022/43022 [00:10<00:00, 4069.13it/s]